A6000

注意，CPU不要跑Float16版本，会慢死的

In [ ]:
!pip install tensorflow-hub sentencepiece jieba --user

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [4]:
!/usr/bin/nvidia-smi

Thu Aug 26 16:13:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  RTX A6000           On   | 00000000:41:00.0 Off |                  Off |
| 30%   34C    P8    21W / 300W |      8MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!ls | grep PanGu || git clone https://github.com/qhduan/PanGu-alpha-tf.git

PanGu-alpha-tf


In [6]:
import sys
sys.path.append('PanGu-alpha-tf')

from tokenization_jieba import JIEBATokenizer

In [7]:
print(tf.__version__)

2.4.1


In [8]:
print(hub.__version__)

0.12.0


In [9]:
def sample(tokenizer, gpt, sentence, number=1, length=20, top_k=50, top_p=0.9, temperature=0.9):
    """
    numbert: 输出句子个数
    length: 输出最大长度
    top_k: 只选择前k个，如果为0则为无限制
    top_p: token的概率排在这以上才有效
    temperature: 温度，如果为1.0则相当于无效
    """
    inputs = tf.constant([tokenizer.encode(sentence)] * number, dtype=tf.int64)
    length = tf.constant(length, dtype=tf.int64)
    ret = gpt.signatures['serving_default'](
        inp=inputs,
        length=length,
        top_k=tf.constant(top_k, tf.int32),
        top_p=tf.constant(top_p, tf.float16),
        temperature=tf.constant(temperature, tf.float16)
    )['output_0']
    return [
        tokenizer.decode([int(x) for x in s]).replace(' ', '')
        for s in ret.numpy()
    ]

In [10]:
tokenizer = JIEBATokenizer(
    'PanGu-alpha-tf/PanGu-Alpha-GPU/panguAlpha_pytorch/megatron/tokenizer/bpe_4w_pcl/vocab.vocab',
    'PanGu-alpha-tf/PanGu-Alpha-GPU/panguAlpha_pytorch/megatron/tokenizer/bpe_4w_pcl/vocab.model')

In [11]:
gpt = hub.load('./pangu-2.6B-tf2-fp16/')

In [18]:
!/usr/bin/nvidia-smi

Thu Aug 26 16:17:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  RTX A6000           On   | 00000000:41:00.0 Off |                  Off |
| 30%   35C    P8    20W / 300W |   8909MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
%%time
q = '''李大嘴：“各回各家，各找各妈！”
白展堂：“'''
ret = sample(tokenizer, gpt, q, 3, 50)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
白展堂:“我有一条祖传的项链。”
李大嘴:“我有一个祖传的红绸包。”
白展堂:“我有一个祖传的玉壶。”
李大嘴:“我有一个祖传
--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“哎,是啊,是啊!”
众人你一言我一语的,说的不亦乐乎。
“小六子,那你说是不是?”
“是啊,是啊!”
--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“各找各妈!”
方子敬:“各找各妈!”
白展堂:“各找各妈!”
方子敬:“各找各妈!”
李大嘴:“
--------------------
CPU times: user 4.35 s, sys: 582 ms, total: 4.93 s
Wall time: 2.78 s


In [14]:
%%time
q = '''关于爱情，李白曾经说过：“'''
ret = sample(tokenizer, gpt, q, 3, 200)
for x in ret:
    print(x)
    print('-' * 20)

关于爱情,李白曾经说过:“山无陵,江水为竭,冬雷震震,夏雨雪,天地合,乃敢与君绝!”如果说,这是他一生中最重要的爱情,那么对于一个女人来说,这也是她最重要的爱情。如果说,这是她一生中最重要的爱情,那么对于一个男人来说,这也是他最重要的爱情。如果说,这是一个女人一生中最重要的爱情,那么对于一个男人来说,这也是他最重要的爱情。我一直觉得,婚姻是两个人的事,要是两个人之间有了爱情,那么这个婚姻肯定是幸福的。因为有了爱情,那么这个婚姻就不再是什么约束,不再是什么约束,你可以随便做任何事情,而且你想做什么就做什么,想怎么样就怎么样。因为有了爱情,你在这世上不管碰到多大的困难,你都能挺过来,
--------------------
关于爱情,李白曾经说过:“夫天地者,万物之逆旅也;光阴者,百代之过客也。而浮生若梦,为欢几何?”人生如幻,我不知道,在这个世界上还有多少值得我为之倾心的,我不知道,我只知道,我一直在寻找的那个人,那就是你。我们俩的爱情如同是一枚璀璨的明珠,从出生起就伴随着整个世界,不管是在地球的哪个角落,只要有阳光的地方,那颗明珠就会熠熠生辉。从我还是一个初中生的时候,就总是幻想着能有一场这样的爱情,我喜欢她,喜欢她的性格,喜欢她的笑容,喜欢她的一切。我坚信她就是我所要找的那个人。但是,我们的爱情,是那么的短暂,那段短短的日子里,我只能从你的笑容里,
--------------------
关于爱情,李白曾经说过:“我喜欢的人,长的并不漂亮,可是,我的心是明亮的。”当爱情来临的时候,我们都希望能够拥有一双属于自己的水晶鞋,爱情是世界上最美好的感情,因为它最纯洁,因为它是美好的,我们应该对此有所了解,才能够真正拥有属于自己的爱情。
一、了解对方的优缺点
1、了解对方的优点是十分重要的,我们可以通过对方的优点来判断自己的爱情是否成功,当我们了解了对方的优点之后,也就能够更好的了解自己的爱情是否是成功的,然后根据这些优点来更好的去判断自己的爱情是否是成功的。
2、了解对方的缺点是十分重要的,我们可以通过对方的缺点来判断自己的爱情是否是失败的,当我们了解了对方的缺点之后,也就能够更好的
--------------------
CPU times: user 17.5 s, sys: 2.21 s, total: 19.7 s
Wall time: 11.1 s


In [17]:
%%time
q = '''关于亲情，达斯·维达曾经说过：“'''
ret = sample(tokenizer, gpt, q, 3, 200)
for x in ret:
    print(x)
    print('-' * 20)

关于亲情,达斯·维达曾经说过:“亲情比什么都重要。”这句话说得一点也没错,就像父母在一起所经历的亲情,虽然短暂,却也重要。我们都知道,当我们的孩子出生时,他们的生命是无法预测的,所以他们需要的是一个能够让他们感觉安全,并且给他们创造快乐的家庭。这个时候,如果爸爸妈妈不在身边,我们就会很难感受到亲情的存在。这个时候,身为父母的我们,要担负起照顾孩子的责任,而不是将孩子的成长责任推卸给别人。有很多的爸爸妈妈说:“我从来不在孩子身边,因为我自己也不知道该怎么去照顾孩子。”“我也不知道该怎么做才对。”“不知道该怎么去安慰孩子。”每当听到这些话,我都觉得很伤心,孩子是我们生命中最重要的人,可是爸爸妈妈不
--------------------
关于亲情,达斯·维达曾经说过:“如果我有足够的钱,我会买一卡车的书、一卡车的衣服、一卡车的鞋子,然后把它们送到我父母和我自己的家里。我知道,这样做对他们有好处,因为他们可以让自己有机会接受更好的教育。我知道,他们可以多花一些钱,为自己购买一些生活必需品,使自己更加富有。我知道,在这些情况下,我会更懂得珍惜他们,而不是一味地索取。”
达斯·维达不仅给了我们很多宝贵的启示,也给了我们很多非常重要的人生智慧。
如果说,达斯·维达给我们上了一堂“金钱哲学”的课,那么,他的另外一个精彩的人生智慧则是把“亲情”看得更加重要。
达斯·维达一直强调的观点是:“金钱是你最值得
--------------------
关于亲情,达斯·维达曾经说过:“如果你想看一个人到死都不肯原谅,那你只能看到他的缺点。”一个人的品性也能影响他的生活方式,有的人性格开朗,有的人则性格内向。如果一个人整天郁郁寡欢,缺乏自信,那么他的生活一定是沉闷乏味的。
其实,每个人都有着属于自己的生活方式,每个人都有自己的兴趣爱好,每个人都有自己的生活习惯,每个人都有着自己的喜怒哀乐。所以,我们不必因为他人的生活方式就去改变他,既然别人的生活方式已经被我们改变了,那我们为什么不从自己身上找找原因呢?
有一个年轻人到了一个偏僻的小镇上。他有一个非常好的朋友,和他在一起的时候,年轻人会觉得很轻松。因为他从来没有想过要改变他人的生活方式,虽然他很
--------------------
CPU times: user 17.3 s, sys: 2.01 s, total: 19.3 s
W

In [20]:
%%time
q = '''菜谱：
鱼香肉丝的做法
'''
ret = sample(tokenizer, gpt, q, 3, 200)
for x in ret:
    print(x)
    print('-' * 20)

菜谱:
鱼香肉丝的做法
原料:
主料:猪里脊肉250克,油菜100克,调料:植物油20克,酱油5克,淀粉(玉米)5克,料酒5克,盐3克,味精2克,胡椒粉2克,香油10克,大葱5克,姜5克,淀粉(豌豆)5克,鸡蛋清30克,
鱼香肉丝的特色
色泽金黄,滑嫩爽口。
鱼香肉丝的做法
原料:
主料:猪里脊肉250克,油菜100克,调料:植物油20克,酱油5克,淀粉(玉米)5克,料酒5克,盐3克,味精2克,胡椒粉2克,香油10克,大葱5克,姜5克,淀粉(豌豆)5克,鸡蛋清30克,
鱼香肉丝的做法
原料:

--------------------
菜谱:
鱼香肉丝的做法
1.把猪肉切成丝,用少许酱油和料酒抓匀腌好;鱼香肉丝中加入盐、糖、姜丝、蒜末、味精、香油,拌匀备用。
2.把菠菜摘洗干净,切成5厘米长的段,用开水焯一下。
3.把水烧开,把菠菜焯一下捞出控干水分。
4.锅里放油,放入腌好的肉丝炒散,加盐、糖、味精、水烧开,再放入菠菜,开锅后淋上香油即可。
小贴士
1.菠菜要焯一下,这样会更入味。
2.菠菜最好先炒一下,这样更入味。
©2019-2020菜谱大全菜谱大全整理发布,大白菜网,大白菜论坛,大白菜网视频大白菜网
本站
--------------------
菜谱:
鱼香肉丝的做法
主料:猪瘦肉200克,木耳(水发)15克,胡萝卜50克。调料:料酒20克,盐3克,味精2克,胡椒粉2克,姜3克,葱5克,植物油50克。
制作方法:
1、将瘦猪肉切成丝,放入碗内,加入精盐、料酒、味精、胡椒粉、姜丝、葱丝拌匀,腌渍15分钟。
2、胡萝卜洗净,切成丝,用沸水焯熟。
3、锅内放入植物油,烧至五成热时,放入肉丝,煸炒至肉丝呈深红色时,加入胡萝卜丝、木耳丝,略炒,烹入料酒,注入鲜汤,用旺火烧开,调入精盐、味精,用湿淀粉勾芡,淋入明油,翻炒均匀后出锅即可。

--------------------
CPU times: user 17.5 s, sys: 2.05 s, total: 19.5 s
Wall time: 11 s


In [22]:
%%time
q = '''他两个搀着手，说说笑笑，转回见了唐僧。唐僧道：“可曾捉得妖怪？”行者道：“那妖怪不奈战，败回钻入水去也。”三藏道：“徒弟，这怪久住于此，他知道浅深。似这般无边的弱水，又没了舟楫，须是得个知水性的，引领引领才好哩。”行者道：“正是这等说。常言道，近朱者赤，近墨者黑。那怪在此，断知水性。我们如今拿住他，且不要打杀，只教他送师父过河，再做理会。”八戒道：“哥哥不必迟疑，让你先去拿他，等老猪看守师父。”行者笑道：“贤弟呀，这桩儿我不敢说嘴。水里勾当，老孙不大十分熟。若是空走，还要捻诀，又念念避水咒，方才走得。不然，就要变化做什么鱼虾蟹鳖之类，我才去得。若论赌手段，凭你在高山云里，干什么蹊跷异样事儿，老孙都会，只是水里的买卖，有些儿榔杭。”八戒道：“老猪当年总督天河，掌管了八万水兵大众，倒学得知些水性，却只怕那水里有什么眷族老小，七窝八代的都来，我就弄他不过，一时不被他捞去耶？”行者道：“你若到他水中与他交战，却不要恋战，许败不许胜，把他引将出来，等老孙下手助你。”八戒道：“言得是，我去耶。”说声去，就剥了青锦直裰，脱了鞋，双手舞钯，分开水路，使出那当年的旧手段，跃浪翻波，撞将进去，径至水底之下，往前正走。'''
ret = sample(tokenizer, gpt, q, 3, 200)
print(q)
print('-' * 20)
print('-' * 20)
for x in ret:
    print(x[len(q):])
    print('-' * 20)

他两个搀着手，说说笑笑，转回见了唐僧。唐僧道：“可曾捉得妖怪？”行者道：“那妖怪不奈战，败回钻入水去也。”三藏道：“徒弟，这怪久住于此，他知道浅深。似这般无边的弱水，又没了舟楫，须是得个知水性的，引领引领才好哩。”行者道：“正是这等说。常言道，近朱者赤，近墨者黑。那怪在此，断知水性。我们如今拿住他，且不要打杀，只教他送师父过河，再做理会。”八戒道：“哥哥不必迟疑，让你先去拿他，等老猪看守师父。”行者笑道：“贤弟呀，这桩儿我不敢说嘴。水里勾当，老孙不大十分熟。若是空走，还要捻诀，又念念避水咒，方才走得。不然，就要变化做什么鱼虾蟹鳖之类，我才去得。若论赌手段，凭你在高山云里，干什么蹊跷异样事儿，老孙都会，只是水里的买卖，有些儿榔杭。”八戒道：“老猪当年总督天河，掌管了八万水兵大众，倒学得知些水性，却只怕那水里有什么眷族老小，七窝八代的都来，我就弄他不过，一时不被他捞去耶？”行者道：“你若到他水中与他交战，却不要恋战，许败不许胜，把他引将出来，等老孙下手助你。”八戒道：“言得是，我去耶。”说声去，就剥了青锦直裰，脱了鞋，双手舞钯，分开水路，使出那当年的旧手段，跃浪翻波，撞将进去，径至水底之下，往前正走。
--------------------
--------------------
那怪急纵身来,八戒赶上去,一手揪住那怪尾巴,一手扯住他衣服,望空一扯,早拽翻在水底下。那怪回头看时,就不见了八戒,只是两只眼睛乱跳。那里知道,只道八戒在水里挣扎,把他绊了一交,那怪便钻入水底去了。八戒笑道:“呆子,这回我不被他捞了,却不被我捉了?”行者道:“你去,我却来。”又道:“你去,我却来。”两个径入水底,将那怪拽到岸上,解下麻绳,捆了⁇棒,拴在岸上,拴在树上。二人下得岸来,爬上岸来,望见岸上一个老妪,拄着拐杖,正倚在树下看门
--------------------
又见那怪立在当中,把他一把提起,就往船里走。八戒在船上,口里念着避水咒,使动着性子,急跑过去,就把那一钯杵望他头上打去。那怪把身一扭,只把杵头一撇,正打在八戒脸上。八戒用手挡架,只叫:“疼,疼!”那怪就把钯一丢,跳上小船,就把八戒抛在水里。这八戒才浮出水面,慌得那水底的船一齐拢来,又把他捞上来。那怪使着神通,使一条黑鱼,一口儿,把八戒一口儿吞下去了。两个搀着手,却似父子一般,进了水寨。
三藏正与八戒斗口,只见那妖怪在水中

In [24]:
%%time
q = '''小郡主格的一笑，掀被下床，笑道：“我穴道早解开了，等了你好久，你怎么到这时候才回来？”
韦小宝奇道：“谁给你解开穴道的？”
小郡主道：“给点了穴道，过得六七个时辰，不用解也自然通了。我扶你上床，我可得走了。”
韦小宝大急，叫道：“不行，不行。你脸上伤痕没好。须得再给你搽药，才好得全。”
小郡主嘻嘻一笑，说道：“你这人真坏，说话老骗人。你几时在我脸上刻花了？倒害得我担心了半天。”
韦小宝问道：“你怎么知道？”
小郡主道：“我早下床来照过镜子，脸上什么也没有。”
'''
ret = sample(tokenizer, gpt, q, 3, 200)
print(q)
print('-' * 20)
print('-' * 20)
for x in ret:
    print(x[len(q):])
    print('-' * 20)

小郡主格的一笑，掀被下床，笑道：“我穴道早解开了，等了你好久，你怎么到这时候才回来？”
韦小宝奇道：“谁给你解开穴道的？”
小郡主道：“给点了穴道，过得六七个时辰，不用解也自然通了。我扶你上床，我可得走了。”
韦小宝大急，叫道：“不行，不行。你脸上伤痕没好。须得再给你搽药，才好得全。”
小郡主嘻嘻一笑，说道：“你这人真坏，说话老骗人。你几时在我脸上刻花了？倒害得我担心了半天。”
韦小宝问道：“你怎么知道？”
小郡主道：“我早下床来照过镜子，脸上什么也没有。”

--------------------
--------------------
韦小宝道:“脸上都刻花了,又怎能有搽药的?”
小郡主道:“只要你脸上没刻花,我就有搽药的,否则我就不搽。”
韦小宝道:“这可使不得。”
小郡主道:“你不搽,我可就搽了。你的脸本来就不搽药的。”
韦小宝道:“你为什么不搽?”
小郡主道:“那你干么......”说到这里,脸一红,脸颊上已泛起了两朵红晕。
韦小宝笑道:“你怕搽药搽得不够,将来搽得够了,我脸上没刻花,你就只好搽得不够。”
小郡主笑道:“你......你笑了
--------------------
韦小宝道:“你怎么又知道的?”
小郡主道:“你脸上的记号,是我在窗前,自己刻上去的。搽药的人一走,我瞧得明明白白。”
韦小宝笑道:“我是个傻瓜,不搽药,也会留下记号的。”
小郡主笑道:“你这人说话太气人了,我不理你了。”站起身来,向外走去。
韦小宝一把拉住,问道:“你去哪里?”
小郡主笑道:“我要去看《四十二章经》,你快跟了我去。”
韦小宝道:“《四十二章经》在哪里?”
小郡主道:“我早在这里跟你说过,你不明白。你想不通的,我就让你想不通。”
--------------------
韦小宝道:“那你自己给自己搽了什么?”
小郡主道:“我不知道,你先给我搽上药,再睡。”
韦小宝只得在床头柜抽屉中找出搽药的药盒,将小郡主脸上所中的毒粉搽上。
小郡主吃了药,一躺上床,便睡了过去。
韦小宝见她脸上果然只印了几行黄豆大小的黄豆大小的小红点,又见她全身雪白,胸脯如雪,心下不忍,也就躺下,酣然入梦。
第二天一早,小郡主服侍他梳洗,小桂子给他捶背。韦小宝笑道:“我不睡觉,还要给你捶背。”小郡主笑道:“我不肯,你这人可真是个懒虫。”

--------------------
CPU ti

In [25]:
%%time
q = '''上联：天地一统，脚踩万千山河
下联：'''
ret = sample(tokenizer, gpt, q, 3, 20)
print(q)
print('-' * 20)
print('-' * 20)
for x in ret:
    print(x[len(q):])
    print('-' * 20)

上联：天地一统，脚踩万千山河
下联：
--------------------
--------------------
日月共辉,足踏万顷大海<eot>如何评价这两个成语?
如何评价“
--------------------
八荒六合,脚踩日月星辰
横批:国泰民安<eot>我不知道要
--------------------
万象更新,脚踩万国衣冠
横批:神州大美<eot>手机版的
--------------------
CPU times: user 1.86 s, sys: 317 ms, total: 2.17 s
Wall time: 1.24 s


In [30]:
%%time
q = '''算术题
1+3=4
3+2=5
1+7='''
ret = sample(tokenizer, gpt, q, 3, 5)
print(q)
print('-' * 20)
print('-' * 20)
for x in ret:
    print(x[len(q):])
    print('-' * 20)

算术题
1+3=4
3+2=5
1+7=
--------------------
--------------------
8
1+9
--------------------
8
7+9
--------------------
8
4+3
--------------------
CPU times: user 571 ms, sys: 57.3 ms, total: 628 ms
Wall time: 348 ms


In [36]:
%%time
q = '''这个人的开放程度是高还是低？
问：很有创意
答：高
问：不喜欢改变
答：低
问：乐于尝试新事物
答：高
问：专注于应对新挑战
答：高
问：不喜欢新事物
答：低
问：乐于思考抽象概念
答：高
问：抵制新想法
答：低
问：想象力不丰富
答：低
问：不喜欢抽象或理论性的概念
答：低
问：喜欢智力挑战
答：高
问：听故事很开心
答：高
问：不会加入少数意见
答：低
问：永远只支持leader
答：低
问：会考虑替代领导者
答：高
问：真的很喜欢哲学
答：高
问：宁愿避免遇到他的敌人
答：低
问：喜欢与人交流
答：'''
ret = sample(tokenizer, gpt, q, 3, 1)
print(q)
print('-' * 20)
print('-' * 20)
for x in ret:
    print(x[len(q):])
    print('-' * 20)

这个人的开放程度是高还是低？
问：很有创意
答：高
问：不喜欢改变
答：低
问：乐于尝试新事物
答：高
问：专注于应对新挑战
答：高
问：不喜欢新事物
答：低
问：乐于思考抽象概念
答：高
问：抵制新想法
答：低
问：想象力不丰富
答：低
问：不喜欢抽象或理论性的概念
答：低
问：喜欢智力挑战
答：高
问：听故事很开心
答：高
问：不会加入少数意见
答：低
问：永远只支持leader
答：低
问：会考虑替代领导者
答：高
问：真的很喜欢哲学
答：高
问：宁愿避免遇到他的敌人
答：低
问：喜欢与人交流
答：
--------------------
--------------------
高
--------------------
高
--------------------
高
--------------------
CPU times: user 231 ms, sys: 66.4 ms, total: 297 ms
Wall time: 233 ms


In [37]:
%%time
q = '''这个人的开放程度是高还是低？
问：很有创意
答：高
问：不喜欢改变
答：低
问：乐于尝试新事物
答：高
问：专注于应对新挑战
答：高
问：不喜欢新事物
答：低
问：乐于思考抽象概念
答：高
问：抵制新想法
答：低
问：想象力不丰富
答：低
问：不喜欢抽象或理论性的概念
答：低
问：喜欢智力挑战
答：高
问：听故事很开心
答：高
问：不会加入少数意见
答：低
问：永远只支持leader
答：低
问：会考虑替代领导者
答：高
问：真的很喜欢哲学
答：高
问：宁愿避免遇到他的敌人
答：低
问：讨厌与人交流
答：'''
ret = sample(tokenizer, gpt, q, 3, 1)
print(q)
print('-' * 20)
print('-' * 20)
for x in ret:
    print(x[len(q):])
    print('-' * 20)

这个人的开放程度是高还是低？
问：很有创意
答：高
问：不喜欢改变
答：低
问：乐于尝试新事物
答：高
问：专注于应对新挑战
答：高
问：不喜欢新事物
答：低
问：乐于思考抽象概念
答：高
问：抵制新想法
答：低
问：想象力不丰富
答：低
问：不喜欢抽象或理论性的概念
答：低
问：喜欢智力挑战
答：高
问：听故事很开心
答：高
问：不会加入少数意见
答：低
问：永远只支持leader
答：低
问：会考虑替代领导者
答：高
问：真的很喜欢哲学
答：高
问：宁愿避免遇到他的敌人
答：低
问：讨厌与人交流
答：
--------------------
--------------------
低
--------------------
低
--------------------
低
--------------------
CPU times: user 252 ms, sys: 21.1 ms, total: 273 ms
Wall time: 191 ms
